In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import tensorflow as tf

In [ ]:
df = pd.read_csv('drive/MyDrive/postings.csv')

In [ ]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower()

df['description'] = df['description'].apply(preprocess_text)
df = df.dropna(subset=['max_salary', 'description'])
df = df[df['max_salary'] > 0]

In [ ]:
df['log_salary'] = np.log1p(df['max_salary'])

In [ ]:
max_words = 10000
max_len = 200
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['description'])
X = tokenizer.texts_to_sequences(df['description'])
X = pad_sequences(X, maxlen=max_len)
y = df['log_salary'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [ ]:
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler.transform(y_test.reshape(-1, 1)).flatten()

In [ ]:
model = Sequential([
    Embedding(max_words, 50, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(
    X_train, y_train_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100
596/596 [==============================] - 11s 16ms/step - loss: 0.9824 - mae: 0.9369 - val_loss: 0.9724 - val_mae: 0.9328
Epoch 2/100
596/596 [==============================] - 9s 15ms/step - loss: 0.8937 - mae: 0.8856 - val_loss: 0.8368 - val_mae: 0.8499
Epoch 3/100
596/596 [==============================] - 11s 19ms/step - loss: 0.7374 - mae: 0.7817 - val_loss: 0.6913 - val_mae: 0.7365
Epoch 4/100
596/596 [==============================] - 9s 14ms/step - loss: 0.6122 - mae: 0.6758 - val_loss: 0.6053 - val_mae: 0.6556
Epoch 5/100
596/596 [==============================] - 5s 9ms/step - loss: 0.5390 - mae: 0.6092 - val_loss: 0.5613 - val_mae: 0.6120
Epoch 6/100
596/596 [==============================] - 5s 9ms/step - loss: 0.4943 - mae: 0.5685 - val_loss: 0.5360 - val_mae: 0.5888
Epoch 7/100
596/596 [==============================] - 7s 11ms/step - loss: 0.4625 - mae: 0.5391 - val_loss: 0.5187 - val_mae: 0.5662
Epoch 8/100
596/596 [==============================] - 5s 9ms/

In [ ]:
loss, mae = model.evaluate(X_test, y_test_scaled, verbose=0)
print(f"\nTest MAE: {mae}")


Test MAE: 0.4297606348991394


In [ ]:
def predict_salary(job_description):
    processed_text = preprocess_text(job_description)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    log_salary = scaler.inverse_transform(prediction)[0][0]
    return np.expm1(log_salary)

In [ ]:
example_job = "Data Scientist with 5 years of experience in machine learning and big data technologies."
predicted_salary = predict_salary(example_job)
print(f"\nPredicted salary: ${predicted_salary:.2f}")

1/1 [==============================] - 0s 98ms/step

Predicted salary: $6847.87


In [ ]:
example_job = "School director based in Missisipi state, 10 years of experience"
predicted_salary = predict_salary(example_job)
print(f"\nPredicted salary: ${predicted_salary:.2f}")

1/1 [==============================] - 0s 20ms/step

Predicted salary: $2652.85


In [ ]:
!pip install joblib

In [ ]:
import joblib

model.save('salary_prediction_model.keras')
joblib.dump(tokenizer, 'tokenizer.joblib')
joblib.dump(scaler, 'scaler.joblib')

print("Model, tokenizer, and scaler have been saved.")

Model, tokenizer, and scaler have been saved.
